# Automatizar adicionamento de vídeos no youtube pelo plugin




### conexão yt

In [1]:
#initialize youtube api

from config import api_key, database_username, database_password, database_ip, database_name
from googleapiclient.discovery import build
import pandas as pd
import json
import re

service = build('youtube','v3', developerKey= api_key)
service

#### Conexão com o banco de dados


In [2]:
import MySQLdb
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect

# database_username= "bc7b6a336b1dc7"
# database_password= "418dbbbb"
# database_ip= "us-cdbr-east-03.cleardb.com"
# database_name= "heroku_55f6de3bde630a5"

database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))



#test to check connection
insp = inspect(database_connection)
print(insp.get_table_names())


['algoritmo1', 'algoritmo2', 'evaluations', 'knex_migrations', 'knex_migrations_lock', 'memento', 'resource', 'users']


In [2]:
#print resource table

import pandas as pd

pd.set_option('display.max_columns', None)

df = pd.read_sql('SELECT * FROM resource', database_connection) 
df1 = pd.read_sql('SELECT * FROM algoritmo1', database_connection)
df2 = pd.read_sql('SELECT * FROM algoritmo2', database_connection)

In [41]:
df1

,id,title,subject,type,description,covarage,relation,external_url,author,contributor,publisher,licence,date_of_publishment,language,audience,format,keywords,technical_requirements,description_of_technical_requirements,last_modification,image,video_link,memento,user_id,resource_id


#### correspondência da API do yt com o formato dublicon core

preencher obrigatoriamente : 


- área do conhecimento

- tipo de material

- descrição do recurso

- autor do recurso

- licença

- onde foi publicado o recurso

- data que o recurso foi criado

- público alvo

- idioma

- palavras-chave

- formato do recurso

#### com correspondencia ao formato dublincore, automatizar adição de vídeos do yt pra db


In [33]:
def add_video(df, response_items, primary_id = -1):
    
    for item in response_items:

        if 'snippet' in item.keys() :
        
            aux = item.get('snippet').get('tags')
            if aux is not None:
                aux = ",".join(aux)
            else:
                aux = "não informado"


            dicionario = { 'type' : 'video', 
         'licence': 'CCBY',
        'description': item.get('snippet').get('description'),
         'subject': 'Multidisciplinar',
         'title': item.get('snippet').get('title'),
                                                              #'subject':   #ENCONTRAAAAAAAAR ,  
         'keywords': aux,
         #'covarage': # não sei o que por,
         'author': item.get('snippet').get('channelTitle'),
         'publisher': 'youtube',                                           #'license':  #ENCONTRAR
         'date_of_publishment': item.get('snippet').get('publishedAt'),
         'language': item.get('snippet').get('defaultAudioLanguage'),
         'audience': 'Outro',
         'format': 'MP4',
         'user_id' : 'a7e17b05-f0bd-4773-a049-90a2ed502635'
        }

            linha = len(df.index)
            df.loc[linha] = dicionario   #certamente não é a melhor prática adicionar linha por linha 
                                         #mas por agora nao vou me preocupar
            df['id'] = df.index
                
                       
            if primary_id == -1:
                
                
                df.loc[linha,'external_url'] ='https://www.youtube.com/watch?v=' + item.get('id')
                df.loc[linha,'video_link'] ='https://www.youtube.com/watch?v=' + item.get('id')
                return df.loc[len(df.index)-1,'id'],df
            
            
            else:
                df.loc[linha,'resource_id'] = primary_id
                
                if item.get('id').get('videoId') != None:
                    df.loc[linha,'external_url'] ='https://www.youtube.com/watch?v=' + item.get('id').get('videoId')
                    df.loc[linha,'video_link'] ='https://www.youtube.com/watch?v=' + item.get('id').get('videoId')
    return df

    
    
    

# criar uma tabela para cada algoritmo com foreign keys para a principal:

In [14]:
from sqlalchemy import text

with database_connection.connect() as connection:

    result = connection.execute(text(   'CREATE TABLE algoritmo1 LIKE resource;'   ))    
    result = connection.execute(text(   'ALTER TABLE algoritmo1 ADD COLUMN resource_id INTEGER REFERENCES resource(id)'   ))
    result = connection.execute(text(   'CREATE TABLE algoritmo2 LIKE resource;'   ))    
    result = connection.execute(text(   'ALTER TABLE algoritmo2 ADD COLUMN resource_id INTEGER REFERENCES resource(id)'   ))

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1050 (42S01): Table 'algoritmo1' already exists
[SQL: CREATE TABLE algoritmo1 LIKE resource;]
(Background on this error at: https://sqlalche.me/e/14/f405)

# Unindo tudo:

In [29]:


def get_related_videos(max_Results, id):
    response = service.search().list(part="snippet",relatedToVideoId= id,type="video", maxResults = max_Results ).execute()
    return response

def search_title(query,max_Results):
    response = service.search().list(q= query,part = 'id, snippet',maxResults = max_Results).execute()
    return response


def add_video_e_sujestoes(video_id,df,df1,df2,qtd):
    
    
    part_string = 'contentDetails,statistics, snippet'
    response = service.videos().list(part = part_string, id= video_id).execute()
    title = response.get('items')[0].get('snippet').get('title')
    
    primary_id, df = add_video(df, [response.get('items')[0]])
    
    #algoritmo1
    sujestoes_response = get_related_videos(qtd, video_id)
    df1 = add_video(df1, sujestoes_response.get('items'),primary_id)
    #algoritmo2
    response = search_title(title,qtd) 
    df2 = add_video(df2,response.get('items'),primary_id)


#add_video_e_sujestoes('ajTTZ7iy6Ls',df,df1,df2)                       
            

In [44]:
lista = ['GQpQha2Mfpg','kCMaqla6Grs','vw_3iMuOvGg','k9WqpQp8VSU','_xf1TMs0ysk']

def tarefa(lista,qtd):
    for item in lista:
        add_video_e_sujestoes(item,df,df1,df2,qtd)

tarefa(lista,10)
   

### função para retornar o video_id de uma external_url


In [8]:
import urllib.parse

def getVideo_id(value):
    """
    Examples:
    - http://youtu.be/SA2iWivDJiE
    - http://www.youtube.com/watch?v=_oPAwA_Udwc&feature=feedu
    - http://www.youtube.com/embed/SA2iWivDJiE
    - http://www.youtube.com/v/SA2iWivDJiE?version=3&amp;hl=en_US
    """
    query = urllib.parse.urlparse(value)
    if query.hostname == 'youtu.be':
        return query.path[1:]
    if query.hostname in ('www.youtube.com', 'youtube.com'):
        if query.path == '/watch':
            p = urllib.parse.parse_qs(query.query)
            return p['v'][0]
        if query.path[:7] == '/embed/':
            return query.path.split('/')[2]
        if query.path[:3] == '/v/':
            return query.path.split('/')[2]
    # fail?
    return None


getVideo_id('https://www.youtube.com/watch?v=MeOUl7ZjJRM')

'MeOUl7ZjJRM'

In [48]:
def sujestoes_alg1ealg2(df,df1,df2,video_resource_id,qtd):
    
    video_link = df[df['id'] == str(video_resource_id)]['external_url'].values[0]
    video_id = getVideo_id(video_link)
    title = df[df['id'] == str(video_resource_id)]['title'].values[0]
    
    
    #algoritmo1
    sujestoes_response = get_related_videos(qtd, video_id)
    df1 = add_video(df1, sujestoes_response.get('items'),video_resource_id)
    #algoritmo2
    response = search_title(title,qtd) 
    df2 = add_video(df2,response.get('items'),video_resource_id)    
    
    
    
sujestoes_alg1ealg2(df,df1,df2,4,10)    
    

In [49]:
df1

,id,title,subject,type,description,covarage,relation,external_url,author,contributor,publisher,licence,date_of_publishment,language,audience,format,keywords,technical_requirements,description_of_technical_requirements,last_modification,image,video_link,memento,user_id,resource_id
0,0,Automatize Qualquer Sistema ou Programa com Py...,Multidisciplinar,video,Quer saber mais sobre o nosso Curso Completo d...,None,None,https://www.youtube.com/watch?v=MeOUl7ZjJRM,Hashtag Programação,None,youtube,CCBY,2021-04-09,,Outro,MP4,não informado,0.0,None,None,None,https://www.youtube.com/watch?v=MeOUl7ZjJRM,None,a7e17b05-f0bd-4773-a049-90a2ed502635,0.0
1,1,MELHOR FORMA DE APRENDER PYTHON (Google Colab ...,Multidisciplinar,video,✅ ALURA COM 10% DE DESCONTO: https://www.alura...,None,None,https://www.youtube.com/watch?v=Gojqw9BQ5qY,Filipe Deschamps,None,youtube,CCBY,2019-09-18,,Outro,MP4,não informado,0.0,None,None,None,https://www.youtube.com/watch?v=Gojqw9BQ5qY,None,a7e17b05-f0bd-4773-a049-90a2ed502635,0.0
2,2,"O QUE EU FARIA, SE TIVESSE QUE COMEÇAR DATA SC...",Multidisciplinar,video,"Nesse vídeo, eu mostro exatamente o PLANEJAMEN...",None,None,https://www.youtube.com/watch?v=VlYDWOfiFuc,Seja Um Data Scientist,None,youtube,CCBY,2020-08-10,,Outro,MP4,não informado,0.0,None,None,None,https://www.youtube.com/watch?v=VlYDWOfiFuc,None,a7e17b05-f0bd-4773-a049-90a2ed502635,1.0
3,3,O que faz e quanto ganha um Analista de Dados,Multidisciplinar,video,O que faz e quanto ganha um Analista de Dados?...,None,None,https://www.youtube.com/watch?v=StWz8HnPnDc,Aprendiz Moderno,None,youtube,CCBY,2020-06-30,,Outro,MP4,não informado,0.0,None,None,None,https://www.youtube.com/watch?v=StWz8HnPnDc,None,a7e17b05-f0bd-4773-a049-90a2ed502635,1.0
4,4,Quais os 10 Temas de Python que Você tem que D...,Multidisciplinar,video,Quer saber mais sobre o nosso Curso Completo d...,None,None,https://www.youtube.com/watch?v=17LOSZ7DOKM,Hashtag Programação,None,youtube,CCBY,2021-05-07,,Outro,MP4,não informado,0.0,None,None,None,https://www.youtube.com/watch?v=17LOSZ7DOKM,None,a7e17b05-f0bd-4773-a049-90a2ed502635,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,60,15 Python Projects in Under 15 Minutes (Code I...,Multidisciplinar,video,View 15 awesome python projects I've worked to...,NaN,NaN,https://www.youtube.com/watch?v=OXi4T58PwdM,Tech With Tim,NaN,youtube,CCBY,2019-06-27T01:19:53Z,None,Outro,MP4,não informado,NaN,NaN,NaN,NaN,https://www.youtube.com/watch?v=OXi4T58PwdM,NaN,a7e17b05-f0bd-4773-a049-90a2ed502635,4.0
61,61,you need to learn Python RIGHT NOW!! // EP 1,Multidisciplinar,video,What I use to learn (the BEST IT training): ht...,NaN,NaN,https://www.youtube.com/watch?v=mRMmlo_Uqcs,NetworkChuck,NaN,youtube,CCBY,2021-08-14T16:27:51Z,None,Outro,MP4,não informado,NaN,NaN,NaN,NaN,https://www.youtube.com/watch?v=mRMmlo_Uqcs,NaN,a7e17b05-f0bd-4773-a049-90a2ed502635,4.0
62,62,Mastering Python - Everything You Need To Know...,Multidisciplinar,video,So what does it take to be a master at python?...,NaN,NaN,https://www.youtube.com/watch?v=p15xzjzR9j0,Tech With Tim,NaN,youtube,CCBY,2020-08-20T00:15:47Z,None,Outro,MP4,não informado,NaN,NaN,NaN,NaN,https://www.youtube.com/watch?v=p15xzjzR9j0,NaN,a7e17b05-f0bd-4773-a049-90a2ed502635,4.0
63,63,Start Automating Your Life Using Python! (File...,Multidisciplinar,video,Sign up to Morning Brew for free today https:/...,NaN,NaN,https://www.youtube.com/watch?v=NCvI-K0Gp90,Internet Made Coder,NaN,youtube,CCBY,2022-05-12T04:26:01Z,None,Outro,MP4,não informado,NaN,NaN,NaN,NaN,https://www.youtube.com/watch?v=NCvI-K0Gp90,NaN,a7e17b05-f0bd-4773-a049-90a2ed502635,4.0


### Enviar para o banco de dados

In [49]:

database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

df.to_sql(con=database_connection, name='resource', if_exists='append',index = False)
df1.to_sql(con=database_connection, name='algoritmo1', if_exists='append',index = False)
df2.to_sql(con=database_connection, name='algoritmo2', if_exists='append',index = False)


-1